https://www.healthline.com/health/lung-consolidation#treatment

In [1]:
from imutils import paths
import cv2
import os
from tqdm import tqdm_notebook
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

/home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/ten

### Data import

In [2]:
imagePaths = list(paths.list_images("dataset"))
data = []
labels = []

#### If imagenet weights are being loaded input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))

In [3]:
img_height=192
img_width = 192
num_classes = 2

In [4]:
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the image, swap color channels, and resize it to be a fixed
	# 192x192 pixels while ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (img_height, img_width))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)


### convert the data and labels to NumPy arrays while scaling the pixel

In [5]:

import numpy as np
data = np.array(data) / 255.0
labels = np.array(labels)


In [6]:
# perform one-hot encoding on the labels
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

### train test split

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=40)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 40
number of test examples = 10
X_train shape: (40, 192, 192, 3)
Y_train shape: (40, 2)
X_test shape: (10, 192, 192, 3)
Y_test shape: (10, 2)


In [8]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# trainAug = ImageDataGenerator(rotation_range=15,fill_mode="nearest")

### base model creation for resnet

In [9]:
from keras import applications

base_model = applications.resnet50.ResNet50(
                            weights= 'imagenet', 
                            include_top=False, 
                            input_shape= (img_height,img_width,3))

Using TensorFlow backend.
W0402 17:03:03.008508 139767167567680 deprecation_wrapper.py:119] From /home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0402 17:03:03.009210 139767167567680 deprecation_wrapper.py:119] From /home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0402 17:03:03.012742 139767167567680 deprecation_wrapper.py:119] From /home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0402 17:03:03.032782 139767167567680 deprecation_wrapper.py:119] From /home/akash/anaconda3/envs/ml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_defa

### Create the model architecture

https://stackoverflow.com/questions/49295311/what-is-the-difference-between-flatten-and-globalaveragepooling2d-in-keras

In [58]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.models import Sequential,Model,load_model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)

model = Model(inputs= base_model.input, outputs= predictions)

In [59]:
from keras.optimizers import SGD, Adam
#sgd = SGD(lr=0.0001, momentum=0.9, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

In [60]:
model.fit(X_train, y_train, epochs = 10, batch_size = 10)

Epoch 1/10
40/40 [==============================] - 9s 214ms/step - loss: 0.4155 - acc: 0.8000
Epoch 2/10
40/40 [==============================] - 1s 13ms/step - loss: 0.0988 - acc: 1.0000
Epoch 3/10
40/40 [==============================] - 0s 11ms/step - loss: 0.0198 - acc: 1.0000
Epoch 4/10
40/40 [==============================] - 0s 11ms/step - loss: 0.0378 - acc: 1.0000
Epoch 5/10
40/40 [==============================] - 1s 13ms/step - loss: 0.0160 - acc: 1.0000
Epoch 6/10
40/40 [==============================] - 0s 12ms/step - loss: 0.0087 - acc: 1.0000
Epoch 7/10
40/40 [==============================] - 0s 12ms/step - loss: 0.0059 - acc: 1.0000
Epoch 8/10
40/40 [==============================] - 0s 11ms/step - loss: 0.0049 - acc: 1.0000
Epoch 9/10
40/40 [==============================] - 0s 11ms/step - loss: 0.0058 - acc: 1.0000
Epoch 10/10
40/40 [==============================] - 0s 11ms/step - loss: 0.0039 - acc: 1.0000


### Accuracy testing by test data

In [61]:
preds = model.evaluate(X_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10/10 [==============================] - 2s 190ms/step
Loss = 0.30771854519844055
Test Accuracy = 0.8999999761581421


In [62]:
from sklearn.metrics import classification_report

predIdxs = model.predict(X_test, batch_size=10)
predIdxs = np.argmax(predIdxs, axis=1)

classification_report(y_test.argmax(axis=1), predIdxs,
	target_names=lb.classes_)

'              precision    recall  f1-score   support\n\n       covid       0.83      1.00      0.91         5\n      normal       1.00      0.80      0.89         5\n\n    accuracy                           0.90        10\n   macro avg       0.92      0.90      0.90        10\nweighted avg       0.92      0.90      0.90        10\n'

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 192, 192, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 198, 198, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 96, 96, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 96, 96, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation